# Structured backward elevations grids

In [ ]:
import os
import numpy as np
from script import moveElev as mvElv

import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

label_size = 7
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size
matplotlib.rc('font', size=6)

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

Get the elevation based on the following known paleo-topography accounting for horizontal displacements. 

First we specify the parameters:

In [ ]:
src_path = 'data'
elev_file = 'paleomap/'
disp_file = 'velocity/velocity_'

# Specify most recent time in Ma BP
startMa = 15
# Specify deepest time in Ma BP
endMa = 20
# Specify paleo-displacements time interval in Ma
dtMa = 1

We now run the main function, in cases where the `dtMa` parameters is not set to 1 then you will have to adjust the number of elevation store in the compressed array file. Same goes with the key values set to z15, z16... that can be adjusted too.

In [ ]:
elevListArray = []
dt = -dtMa*1.e6
timeframe = np.arange(startMa,endMa,dtMa)
outputfile = 'backward-elev'+str(startMa)+'-'+str(endMa)
ofile = os.path.join(src_path,outputfile)

# dispListArray = [] # Not necessary...

for k in range(len(timeframe)):
    
    dtime = timeframe[k]
    dispfile = os.path.join(src_path, disp_file+str(dtime)+".00Ma.xy")
    
    if k == 0:
        elevfile = os.path.join(src_path, elev_file+str(timeframe[k])+"Ma.nc")
        mve = mvElv.moveElev(elevfile=elevfile, dispfile=dispfile, 
                             elev=None, erodep=None, scotese=True)
    else:
        mve.dispfile = dispfile
        mve.elev = mve.new_elev.copy()
        mve.getDisplacements()
        
    mve.applyDisplacements(dt)
    print('Computed elevation at',timeframe[k],'Ma before applying the',dtime,'Ma displacements\n')
    elevListArray.append(mve.new_elev)
    
#     dispListArray.append(mve.disps) # Not necessary...

# # Save the backward elevations as a compressed numpy file
# np.savez_compressed(ofile, z15=elevListArray[0].flatten(), z16=elevListArray[1].flatten(),
#                    z17=elevListArray[2].flatten(), z18=elevListArray[3].flatten(), 
#                    z19=elevListArray[4].flatten())

# Not necessary
# np.savez_compressed('paleo-disp15-20', d15=dispListArray[0], d16=dispListArray[1],
#                    d17=dispListArray[2], d18=dispListArray[3], 
#                    d19=dispListArray[4])

print('Backward paleo-elevations grids have been saved in',ofile+'.npz','\n')

In [ ]:
# Save the backward elevations as a compressed numpy file
np.savez_compressed(ofile, 
                    z15=elevListArray[0].flatten(), 
                    z16=elevListArray[1].flatten(),
                    z17=elevListArray[2].flatten(), 
                    z18=elevListArray[3].flatten(), 
                    z19=elevListArray[4].flatten()
                   )

Check the created backward elevations...

In [ ]:
# for k in range(len(elevListArray)):
    
#     fig = plt.figure(figsize=(10,8))
#     ax = fig.add_subplot(111)
#     ax.imshow(np.flipud(elevListArray[k]), extent=(-180, 180, -90, 90)) #, #vmin=-10, vmax=10, 
#     ax.set(xlabel='Longitude', ylabel='Latitude', yticks=np.arange(-90,120,30), xticks=np.arange(-180,180,30))
#     ax.minorticks_on()

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
ax.imshow(np.flipud(elevListArray[-1]-elevListArray[0]), extent=(-180, 180, -90, 90), vmin=-5000, vmax=5000, cmap=cm.RdBu) 
ax.set(xlabel='Longitude', ylabel='Latitude', yticks=np.arange(-90,120,30), xticks=np.arange(-180,180,30))
ax.minorticks_on()